# 如何为聊天机器人添加工具

:::info 先决条件

本指南假设您熟悉以下概念：

- [聊天机器人](/docs/concepts/messages)
- [代理](/docs/tutorials/agents)
- [聊天历史记录](/docs/concepts/chat_history)

:::

本节将介绍如何创建会话式代理：能够使用工具与其他系统和 API 进行交互的聊天机器人。

:::note

此操作指南以前是使用 [RunnableWithMessageHistory](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html) 构建聊天机器人。您可以在 [v0.2 文档](https://python.langchain.com/v0.2/docs/how_to/chatbots_tools/) 中找到此版本的指南。

自 LangChain v0.3 版本发布以来，我们建议 LangChain 用户利用 [LangGraph 持久化](https://langchain-ai.github.io/langgraph/concepts/persistence/) 将 `memory` 集成到新的 LangChain 应用程序中。

如果您的代码已依赖于 `RunnableWithMessageHistory` 或 `BaseChatMessageHistory`，则**无需**进行任何更改。我们目前不打算弃用此功能，因为它适用于简单的聊天应用程序，并且任何使用 `RunnableWithMessageHistory` 的代码将继续按预期工作。

有关更多详细信息，请参阅[如何迁移到 LangGraph Memory](/docs/versions/migrating_memory/)。
:::

## 设置

在本指南中，我们将使用一个 [工具调用代理](https://langchain-ai.github.io/langgraph/concepts/agentic_concepts/#tool-calling-agent)，该代理只包含一个用于搜索网络的工具。默认将由 [Tavily](/docs/integrations/tools/tavily_search) 提供支持，但您可以将其替换为任何类似的工具。本节其余部分将假定您使用的是 Tavily。

您需要访问 Tavily 网站进行[注册账户](https://tavily.com/)，并安装以下软件包：

In [ ]:
%pip install --upgrade --quiet langchain-openai tavily-python langgraph

import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API Key:")

您还需要将 OpenAI 密钥设置为 `OPENAI_API_KEY`，并将 Tavily API 密钥设置为 `TAVILY_API_KEY`。

## 创建一个 Agent

我们的最终目标是创建一个 Agent，使其能够与用户进行对话式交互，并在需要时查找信息。

首先，让我们初始化 Tavily 和一个支持工具调用的 OpenAI [chat model](/docs/concepts/chat_models/)：

In [1]:
from langchain_openai import ChatOpenAI
from langchain_tavily import TavilySearch

tools = [TavilySearch(max_results=10, topic="general")]

# Choose the LLM that will drive the agent
# Only certain models support this
model = ChatOpenAI(model="gpt-4o-mini")

为了让我们的代理具有对话能力，我们还可以指定一个提示（prompt）。下面是一个例子：

In [2]:
prompt = (
    "You are a helpful assistant. "
    "You may not need to use tools for every query - the user may just want to chat!"
)

太棒了！现在，让我们使用 LangGraph 的预构建模块 [create_react_agent](https://langchain-ai.github.io/langgraph/reference/prebuilt/#langgraph.prebuilt.chat_agent_executor.create_react_agent) 来组装我们的代理，该模块允许你创建一个 [tool-calling agent](https://langchain-ai.github.io/langgraph/concepts/agentic_concepts/#tool-calling-agent)：

In [3]:
from langgraph.prebuilt import create_react_agent

# prompt allows you to preprocess the inputs to the model inside ReAct agent
# in this case, since we're passing a prompt string, we'll just always add a SystemMessage
# with this prompt string before any other messages sent to the model
agent = create_react_agent(model, tools, prompt=prompt)

## 运行 Agent

既然我们已经设置好了 Agent，就来尝试与它交互吧！它可以处理一些无需查询的简单问题：

In [4]:
from langchain_core.messages import HumanMessage

agent.invoke({"messages": [HumanMessage(content="I'm Nemo!")]})

{'messages': [HumanMessage(content="I'm Nemo!", additional_kwargs={}, response_metadata={}, id='40b60204-1af1-40d4-b6a7-b845a2281dd6'),
  AIMessage(content='Hi Nemo! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 795, 'total_tokens': 806, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BjsUwqprT2mVdjqu1aaSm1jVVWYVz', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--08282ec6-6d3e-4495-b004-b3b08f3879c3-0', usage_metadata={'input_tokens': 795, 'output_tokens': 11, 'total_tokens': 806, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}

或者，如有需要，它可以使用传入的搜索工具来获取最新信息：

In [5]:
agent.invoke(
    {
        "messages": [
            HumanMessage(
                content="What is the current conservation status of the Great Barrier Reef?"
            )
        ],
    }
)

{'messages': [HumanMessage(content='What is the current conservation status of the Great Barrier Reef?', additional_kwargs={}, response_metadata={}, id='5240955c-d842-408d-af3d-4ee74db29dbd'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_W37BFkNuZlJu9US1Tl71xpiX', 'function': {'arguments': '{"query":"current conservation status of the Great Barrier Reef","time_range":"year","topic":"general"}', 'name': 'tavily_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 804, 'total_tokens': 836, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BjsV6EJ7F1vDipoG4dpEiBRZvuTLo', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None

## 对话式回复

由于我们的提示中包含聊天历史消息的占位符，我们的代理还可以考虑之前的交互，并像标准聊天机器人一样以对话方式响应：

In [6]:
from langchain_core.messages import AIMessage, HumanMessage

agent.invoke(
    {
        "messages": [
            HumanMessage(content="I'm Nemo!"),
            AIMessage(content="Hello Nemo! How can I assist you today?"),
            HumanMessage(content="What is my name?"),
        ],
    }
)

{'messages': [HumanMessage(content="I'm Nemo!", additional_kwargs={}, response_metadata={}, id='8a67dea0-acd8-40f9-8c28-292c5f81c05f'),
  AIMessage(content='Hello Nemo! How can I assist you today?', additional_kwargs={}, response_metadata={}, id='92a2533e-5c62-4cbe-80f1-302f5f1caf28'),
  HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}, id='efa8c3d3-86d7-428f-985e-a3aadd6504bc'),
  AIMessage(content='Your name is Nemo!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 818, 'total_tokens': 824, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BjsVIf5MX5jXUEjYCorT5bWYzc7iu', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None

如果愿意，你还可以为 LangGraph 代理添加内存来管理消息历史记录。我们将其重新声明如下：

In [7]:
from langgraph.checkpoint.memory import MemorySaver

# highlight-start
memory = MemorySaver()
agent = create_react_agent(model, tools, prompt=prompt, checkpointer=memory)
# highlight-end

In [8]:
agent.invoke(
    {"messages": [HumanMessage("I'm Nemo!")]},
    config={"configurable": {"thread_id": "1"}},
)

{'messages': [HumanMessage(content="I'm Nemo!", additional_kwargs={}, response_metadata={}, id='31c2249a-13eb-4040-b56d-0c8746fa158e'),
  AIMessage(content='Hello, Nemo! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 795, 'total_tokens': 807, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BjsVRB0FItvtPawTTIAjNwgmlQFFw', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--a9703ca1-de4c-4f76-b622-9683d86ca777-0', usage_metadata={'input_tokens': 795, 'output_tokens': 12, 'total_tokens': 807, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}

然后，如果我们重新运行我们的包装代理执行器：

In [9]:
agent.invoke(
    {"messages": [HumanMessage("What is my name?")]},
    config={"configurable": {"thread_id": "1"}},
)

{'messages': [HumanMessage(content="I'm Nemo!", additional_kwargs={}, response_metadata={}, id='31c2249a-13eb-4040-b56d-0c8746fa158e'),
  AIMessage(content='Hello, Nemo! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 795, 'total_tokens': 807, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BjsVRB0FItvtPawTTIAjNwgmlQFFw', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--a9703ca1-de4c-4f76-b622-9683d86ca777-0', usage_metadata={'input_tokens': 795, 'output_tokens': 12, 'total_tokens': 807, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
  HumanMessage(co

此 [LangSmith 跟踪](https://smith.langchain.com/public/9e6b000d-08aa-4c5a-ac83-2fdf549523cb/r) 展示了幕后发生的情况。

## 进一步阅读

要了解更多关于如何构建代理的信息，请参阅这些 [LangGraph](https://langchain-ai.github.io/langgraph/) 指南：

* [代理概念指南](https://langchain-ai.github.io/langgraph/concepts/agentic_concepts/)
* [代理教程](https://langchain-ai.github.io/langgraph/tutorials/multi_agent/multi-agent-collaboration/)
* [create_react_agent](https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/)

要了解更多关于工具使用的问题，您还可以查看 [此用例部分](/docs/how_to#tools)。